# Using ESMFold to Predict Protein Structures on SageMaker

NOTE: The authors recommend running this notebook in Amazon SageMaker Studio with the following environment settings:  
* **PyTorch 1.13 Python 3.9 CPU-optimized** image  
* **Python 3** kernel  
* **ml.r5.xlarge** instance type  

For improved performance, you may also use the **PyTorch 1.13 Python 3.9 GPU-optimized** image and a **ml.g4dn.2xlarge** instance type.

---

Understanding the structure of proteins like antibodies is important for understanding their function. However, it can be difficult and expensive to do this in a laboratory. Recently AI-driven protein folding algorithms have enabled biologists to predict these structures from their aminio acid sequences instead.

In this notebook, we will use the [ESMFold](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1) protein folding algorithm to predict the structure of Herceptin (Trastuzumab), an important breast cancer therapy. Herceptin is a [monoclonal antibody](https://www.cancer.org/treatment/treatments-and-side-effects/treatment-types/immunotherapy/monoclonal-antibodies.html) (mAb) that binds to the HER2 receptor, inhibiting cancer cell growth. The following diagram shows several of the common elements of monoclonal antibodies.

![A diagram of the major structural elements of an antibody](img/antibody.png)

In this notebook, we'll focus on predicting the structure of the heavy chain region.

In [2]:
%pip install -U -q -r notebook-requirements.txt --disable-pip-version-check

Note: you may need to restart the kernel to use updated packages.


## 1. Download and Visualize the Experimentally-Determined Herceptin Protein Structure

In [24]:
from Bio.PDB import PDBList, MMCIFParser
import os
import py3Dmol
from prothelpers.structure import atoms_to_pdb
import warnings

target_id = "1N8Z"

if not os.path.isdir("data"):
    os.mkdir("data")

pdbl = PDBList()
filename = pdbl.retrieve_pdb_file(target_id, pdir="data", file_format="mmCif")
parser = MMCIFParser()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    structure_1N8Z = parser.get_structure(target_id, filename)
pdb_string = atoms_to_pdb(structure_1N8Z[0])

view = py3Dmol.view(width=600, height=400)
view.addModel(pdb_string)
view.setStyle({"chain": "A"}, {"cartoon": {"color": "orange", "opacity": 0.5}})
view.setStyle({"chain": "B"}, {"cartoon": {"color": "blue"}})
view.setStyle({"chain": "C"}, {"cartoon": {"color": "green", "opacity": 0.5}})
view.zoomTo()
view.show()

Structure exists: 'data/1n8z.cif' 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In the image above, the light chain (A) is in orange, the heavy chain (B) is in blue, and the HER2 antigen (chain C) is in green. In this notebook, we will use ESMFold to predict the structure of chain B from it's amino acid sequence. Then, we will compare the prediction to the experimentally-determined structure shown above.

Extract the structure and sequence of chain B for later use.

In [3]:
from prothelpers.structure import get_aa_seq

experimental_structure = atoms_to_pdb(structure_1N8Z[0]["B"])
with open("data/experimental.pdb", "w") as f:
    f.write(experimental_structure)

experimental_sequence = get_aa_seq(structure_1N8Z[0]["B"])

## 2. Make an In-Notebook ESMFold Prediction
We'll use the ESMFold model to predict the structure of the heavy chain and compare it to the experimental result. First, we load the pre-trained ESMFold model and tokenizer from HuggingFace Hub. This will take about one minute.

In [4]:
from transformers import AutoTokenizer, EsmForProteinFolding

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/esmfold_v1"
)
model = EsmForProteinFolding.from_pretrained(
    "facebook/esmfold_v1", low_cpu_mem_usage=True
)

Some weights of EsmForProteinFolding were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


If using a GPU, we can switch to half-precision and use other optimizations to improve performance. We can also use a CPU for lower cost, but slower performance.

In [5]:
import torch

if torch.cuda.is_available(): 
    device = torch.device("cuda")
    model.esm = model.esm.half()
    torch.backends.cuda.matmul.allow_tf32 = True
else:
    print("CUDA not detected. Using CPU-only parameters")
    device = torch.device("cpu")
    model.esm = model.esm.float()
    torch.backends.cuda.matmul.allow_tf32 = False

model = model.to(device)
model.trunk.set_chunk_size(64)

CUDA not detected. Using CPU-only parameters


Next, we tokenize the sequence to convert it into a numerical format that ESMFold can use for prediction

In [6]:
tokenized_input = tokenizer(
    [experimental_sequence], return_tensors="pt", add_special_tokens=False
)["input_ids"]
tokenized_input = tokenized_input.to(device)

print(f"The human-readable sequence is {experimental_sequence}")
print(f"The tokenized representation of the sequences is {tokenized_input}")

The human-readable sequence is EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP
The tokenized representation of the sequences is tensor([[ 6, 19,  5, 10, 19,  6, 15,  7,  7,  7, 10, 19,  5, 14,  7,  7, 15, 10,
          1, 10, 15,  4,  0,  0, 15,  7, 13,  2,  9, 11,  3, 16, 18,  9,  8, 17,
         19,  1,  5,  0, 14,  7, 11,  7, 10,  6, 17, 19,  0,  1,  9, 18, 14, 16,
          2,  7, 18, 16,  1, 18,  0,  3, 15, 19, 11,  7,  1, 13, 16,  9, 15,  0,
          3, 16, 15, 11,  2, 16,  0, 18, 10,  5, 12,  2, 15, 10,  1,  0,  6,  3,
         16,  0, 19, 18, 18,  4, 15,  1, 17,  7,  7,  3,  7, 13, 18,  0, 12,  3,
         18, 17,  7,  5,  7, 16, 10, 19, 16, 19, 15, 15,  0, 15, 16, 11,  7, 14,
         15, 19, 13, 14, 10,  0, 14, 15, 15, 11, 15, 16, 15,  7,  7, 16,  0,  0,
         10,  7,  4, 10, 19, 11,  3, 18, 13, 14,  6

Finally, we submit the tokenized sequence to the ESMFold model to predict the 3D structure. This will take about three minutes on a non-accelerated instance type and several seconds on an accelerated instance.

In [8]:
%%time
print(f"Predicting the structure of protein sequence {experimental_sequence}")
with torch.no_grad():
    notebook_prediction = model.infer_pdb(experimental_sequence)

with open("data/prediction.pdb", "w") as f:
    f.write(notebook_prediction)

if torch.cuda.is_available(): 
    torch.cuda.empty_cache()
    
print(f"Prediction complete")

Predicting the structure of protein sequence EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP
Prediction complete
CPU times: user 6min 13s, sys: 608 ms, total: 6min 14s
Wall time: 3min 7s


We can determine the accuracy of our prediction by comparing our prediction to the experimental structure.

In [9]:
from prothelpers.usalign import tmscore

tmscore("data/prediction.pdb", "data/experimental.pdb", pymol="data/superimposed")

,PDBchain1,PDBchain2,TM1,TM2,RMSD,ID1,ID2,IDali,L1,L2,Lali
0,data/prediction.pdb:A,data/experimental.pdb:B,0.802,0.802,3.04,0.918,0.918,0.922,220,220,219


A TM-Score of 0.8 isn't perfect, but good enough for analytical tasks like ligand-binding analysis. Let's visualize how the experimentally-determined and predicted structure align.

In [10]:
view = py3Dmol.view(width=600, height=400)

with open("data/experimental.pdb") as ifile:
    superimposed = "".join([x for x in ifile])
view.addModel(experimental_structure)

with open("data/superimposed.pdb") as ifile:
    superimposed = "".join([x for x in ifile])
view.addModel(superimposed)

view.setStyle({"model": 0}, {"cartoon": {"color": "blue"}})
view.setStyle({"model": 1}, {"cartoon": {"color": "red"}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The two structures show a high, but not perfect, degree of overlap. Protein structure predictions is a rapidly-evolving field and many research teams are developing ever-more accurate algorithms!

## 3. Deploy ESMFold as a SageMaker Inference API

Running model inference in a notebook is fine for experimentation, but what if you need to integrate your model with an application? Or an MLOps pipeline? In this case, a better option is to deploy your model as an API endpoint. In this example, we'll deploy ESMFold as a real-time inference endpoint on an accelerated instance.

In [11]:
import sagemaker
import boto3
import os

boto_session = boto3.Session()
sm_sess = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
region = sm_sess.boto_region_name
bucket = sm_sess.default_bucket()
prefix = "sagemaker/ESMfold"
role = sagemaker.get_execution_role()

if not os.path.isdir("code"):
    os.mkdir("code")

The pre-built SageMaker container for HuggingFace makes it easy to deploy deep learning models for common tasks. However, to predict protein structures we need to define a custom `inference.py` script to load the model, run the prediction, and format the output.

In [12]:
%%writefile code/inference.py

import json
import numpy as np
import os
import torch
import traceback
import transformers
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from typing import Any, Dict, List

MODEL_NAME = "facebook/esmfold_v1"

def model_fn(model_dir: str) -> Dict[str, Any]:
    """ Load the model artifact """

    try:
        model_path = os.path.join(model_dir, "esmfold_v1")
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
        model = transformers.EsmForProteinFolding.from_pretrained(model_path, low_cpu_mem_usage=True)

        if torch.cuda.is_available():
            model.to("cuda")
            model.esm = model.esm.half()
            torch.backends.cuda.matmul.allow_tf32 = True
            model.trunk.set_chunk_size(64)
        else:
            model.to("cpu")
            model.esm = model.esm.float()
            model.trunk.set_chunk_size(64)

        return tokenizer, model
    except Exception as e:
        traceback.print_exc()
        raise e
        
def input_fn(request_body: str, request_content_type: str = "text/csv") -> List[str]:
    """ Process the request """

    print(request_content_type)
    
    if request_content_type == "text/csv":
        sequence = request_body
        print("Input protein sequence: ", sequence)
        return sequence
    elif request_content_type == "application/json":
        sequence = json.loads(request_body)
        print("Input protein sequence: ", sequence)
        return sequence
    else:
        raise ValueError("Unsupported content type: {}".format(request_content_type))        

def predict_fn(input_data: List, tokenizer_model: tuple) -> np.ndarray:
    """ Run the prediction """
    
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        esm_tokenizer, esm_model = tokenizer_model
        tokenized_input = esm_tokenizer(
            input_data, return_tensors="pt", add_special_tokens=False
        )["input_ids"].to(device)

        with torch.no_grad():
            output = esm_model(tokenized_input)
            return output
    except Exception as e:
        traceback.print_exc()
        raise e
        
def output_fn(outputs: str, response_content_type: str = "text/csv"):
    """Transform the prediction into a pdb-formatted string"""
    
    final_atom_positions = atom14_to_atom37(outputs["positions"][-1], outputs)
    outputs = {k: v.to("cpu").numpy() for k, v in outputs.items()}
    final_atom_positions = final_atom_positions.cpu().numpy()
    final_atom_mask = outputs["atom37_atom_exists"]
    pdbs = []
    for i in range(outputs["aatype"].shape[0]):
        aa = outputs["aatype"][i]
        pred_pos = final_atom_positions[i]
        mask = final_atom_mask[i]
        resid = outputs["residue_index"][i] + 1
        pred = OFProtein(
            aatype=aa,
            atom_positions=pred_pos,
            atom_mask=mask,
            residue_index=resid,
            b_factors=outputs["plddt"][i],
            chain_index=outputs["chain_index"][i] if "chain_index" in outputs else None,
        )
        pdbs.append(to_pdb(pred))
        
    if response_content_type == "text/csv":
        return pdbs
    elif response_content_type == "application/json":
        return json.dumps(pdbs)
    else:
        raise ValueError("Unsupported content type: {}".format(response_content_type))

Writing code/inference.py


We can also define additional packages needed by our inference code in a `requirements.txt` file.

In [13]:
%%writefile code/requirements.txt

transformers==4.24.0
accelerate==0.17.0

Writing code/requirements.txt


For this example, we use a model artifact previously downloaded from HuggingFace hub and uploaded to S3.

In [14]:
from sagemaker.huggingface import HuggingFaceModel
from datetime import datetime

public_model_url = "s3://aws-hcls-ml/workshop/esmfold/esmfold_v1.tar.gz"

huggingface_model = HuggingFaceModel(
    model_data=public_model_url,
    name=f"emsfold-v1-model-" + datetime.now().strftime("%Y%m%d%s"),
    transformers_version="4.17",
    pytorch_version="1.10",
    py_version="py38",
    role=role,
    source_dir="code",
    entry_point="inference.py",
)

Deploy the model to a real-time inference endpoint hosted on an accelerated instance type. This may take 10 minutes or more depending on the availability of ml.g4dn instances in your region.

In [15]:
%%time

rt_predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.2xlarge",
    endpoint_name=f"emsfold-v1-rt-endpoint-" + datetime.now().strftime("%Y%m%d%s"),
)

23/05/09 13:39:18 | INFO | sagemaker | Creating model with name: emsfold-v1-model-202305091683638908
23/05/09 13:39:19 | INFO | sagemaker | Creating endpoint-config with name emsfold-v1-rt-endpoint-202305091683638923
23/05/09 13:39:19 | INFO | sagemaker | Creating endpoint with name emsfold-v1-rt-endpoint-202305091683638923
----------!CPU times: user 8min 38s, sys: 1min 23s, total: 10min 1s
Wall time: 16min 7s


Submit the chain B sequence to the endpoint and print the first few rows of the predicted structure file.

In [16]:
endpoint_prediction = rt_predictor.predict(experimental_sequence)[0]
print(endpoint_prediction[:900])

PARENT N/A
ATOM      1  N   GLU A   1      14.578 -19.953   1.470  1.00  0.83           N  
ATOM      2  CA  GLU A   1      13.166 -19.595   1.577  1.00  0.84           C  
ATOM      3  C   GLU A   1      12.737 -18.693   0.423  1.00  0.86           C  
ATOM      4  CB  GLU A   1      12.886 -18.906   2.915  1.00  0.80           C  
ATOM      5  O   GLU A   1      13.417 -17.715   0.106  1.00  0.83           O  
ATOM      6  CG  GLU A   1      11.406 -18.694   3.200  1.00  0.71           C  
ATOM      7  CD  GLU A   1      11.141 -18.042   4.548  1.00  0.68           C  
ATOM      8  OE1 GLU A   1      12.108 -17.805   5.307  1.00  0.68           O  
ATOM      9  OE2 GLU A   1       9.958 -17.767   4.847  1.00  0.61           O  
ATOM     10  N   VAL A   2      11.678 -19.063  -0.258  1.00  0.87           N  
ATOM     11  CA  VAL A   2      11.207 -18.309  -1.415  1.00  0.87           C 


Finally, save the predicted structure to a file and then compare the two ESMFold predictions.

In [17]:
from prothelpers import usalign

with open("data/endpoint_prediction.pdb", "w") as f:
    f.write(endpoint_prediction)

usalign.tmscore(
    "data/endpoint_prediction.pdb", "data/prediction.pdb", pymol="data/esm_superimposed"
)

,PDBchain1,PDBchain2,TM1,TM2,RMSD,ID1,ID2,IDali,L1,L2,Lali
0,data/endpoint_prediction.pdb:A,data/prediction.pdb:A,1.0,1.0,0.04,1.0,1.0,1.0,220,220,220


In [18]:
view2 = py3Dmol.view(width=600, height=400)

with open("data/prediction.pdb") as ifile:
    prediction = "".join([x for x in ifile])
view2.addModel(prediction)

with open("data/esm_superimposed.pdb") as ifile:
    esm_superimposed = "".join([x for x in ifile])
view2.addModel(esm_superimposed)

view2.setStyle({"model": 0}, {"cartoon": {"color": "red"}})
view2.setStyle({"model": 1}, {"cartoon": {"color": "blue"}})
view2.zoomTo()
view2.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The notebook and endpoint prediction are identical.

In [22]:
endpoint_name = rt_predictor.endpoint_name
%store endpoint_name

Stored 'endpoint_name' (str)


## 4. Clean Up

In [19]:
## Uncomment these lines to remove the SageMaker endpoint and model data
## and halt further charges

# # Delete endpoints
# try:
#     rt_predictor.delete_endpoint()
# except:
#     pass

# # Delete all S3 objects
# bucket = boto_session.resource("s3").Bucket(bucket)
# bucket.objects.filter(Prefix=prefix).delete()

23/03/22 20:44:56 | INFO | sagemaker | Deleting endpoint configuration with name: emsfold-v1-rt-endpoint-202303221679516912
23/03/22 20:44:56 | INFO | sagemaker | Deleting endpoint with name: emsfold-v1-rt-endpoint-202303221679516912


0